In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random as rand
from tqdm import tqdm
import re
import string

In [58]:
video_games = {'game':[],'release':[]}
URL = 'https://www.metacritic.com/browse/games/score/metascore/year/pc/filtered?year_selected=2022&distribution=&sort=desc&view=condensed&page=0'

user_agent = {'User-agent': 'Mozilla/5.0'}
response = requests.get(URL, headers = user_agent)
soup = BeautifulSoup(response.text, 'html.parser')

pages = int(soup.find('a', class_='page_num').text)
word_pattern = '[A-Z][a-z]+ \d+, \d\d\d\d'

curr_page = 0
while(curr_page <= pages):
    URL = 'https://www.metacritic.com/browse/games/score/metascore/year/pc/filtered?year_selected=2022&distribution=&sort=desc&view=condensed&page='+ str(curr_page)
    response = requests.get(URL, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for entry in soup.find_all('td', class_='details'):
        title = entry.find('a', class_='title')['href']
        title = title[9:]
        video_games['game'].append(title)
        video_games['release'].append(entry.find_all(text=re.compile(word_pattern)))
        
    curr_page += 1
    
video_games

{'game': ['persona-5-royal',
  'elden-ring',
  'dwarf-fortress',
  'god-of-war',
  'i-was-a-teenage-exocolonist',
  'chained-echoes',
  'the-stanley-parable-ultra-deluxe',
  'the-legend-of-heroes-trails-from-zero',
  'neon-white',
  'cuphead-in-the-delicious-last-course',
  'norco',
  'ai-the-somnium-files---nirvana-initiative',
  'rogue-legacy-2',
  'marvels-spider-man-miles-morales',
  'uncharted-legacy-of-thieves-collection',
  'pentiment',
  'immortality',
  'olliolli-world',
  'vampire-survivors',
  'final-fantasy-vi-pixel-remaster',
  'marvels-spider-man-remastered',
  'return-to-monkey-island',
  'monster-hunter-rise-sunbreak',
  'monster-hunter-rise',
  'total-war-warhammer-iii',
  'destiny-2-the-witch-queen',
  'tunic',
  'death-stranding-directors-cut',
  'tactics-ogre-reborn',
  'olliolli-world-void-riders',
  'red-matter-2',
  'the-case-of-the-golden-idol',
  'infernax',
  'teenage-mutant-ninja-turtles-shredders-revenge',
  'gordian-quest',
  'wylde-flowers',
  'neverawake'

In [61]:
video_games = pd.DataFrame(video_games)
video_games.to_csv('video_games.csv', index=False)
#video_games = pd.read_csv('video_games.csv')
video_games.head(20)

,game,release
0,persona-5-royal,"[October 21, 2022]"
1,elden-ring,"[February 25, 2022]"
2,dwarf-fortress,"[December 6, 2022]"
3,god-of-war,"[January 14, 2022]"
4,i-was-a-teenage-exocolonist,"[August 25, 2022]"
5,chained-echoes,"[December 8, 2022]"
6,the-stanley-parable-ultra-deluxe,"[April 27, 2022]"
7,the-legend-of-heroes-trails-from-zero,"[September 27, 2022]"
8,neon-white,"[June 16, 2022]"
9,cuphead-in-the-delicious-last-course,"[June 30, 2022]"


In [63]:
review_dict = {'game':[],'username':[], 'date':[],'score':[],'review':[], 'helpful':[]}

for video_game in tqdm(video_games['game']):
    page = 0
    lastpage = 1
    while page < lastpage:
        time.sleep(rand.uniform(2,4)) 
        URL = 'https://www.metacritic.com/game/pc/'+video_game+'/user-reviews?sort-by=date&num_items=100&page='+str(page)
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response = requests.get(URL, headers=user_agent)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if (soup.find('li', class_='page last_page') and lastpage == 1):
            lastpage = int(soup.find('li', class_='page last_page').find('a').text)
        page += 1
        reviews = soup.find('div', class_='body product_reviews')

        for review in reviews.find_all('div', class_='review_top review_top_l'):
            if (soup.find('div', class_='msg msg_no_reviews')):
                break
            
            ups = review.find('span', class_='total_ups')
            total = review.find('span', class_='total_thumbs')
            if (int(total.text) == 0):
                review_dict['helpful'].append(0)
            else:
                review_dict['helpful'].append(int(ups.text)/int(total.text))
            
            review_dict['game'].append(video_game)
            review_dict['username'].append(review.find('div', class_='name').text.replace('\n',''))
            review_dict['date'].append(review.find('div', class_='date').text)
            review_dict['score'].append(review.find('div', class_='review_grade').text.replace('\n',''))

            if (review.find('span', class_='blurb blurb_expanded')):
                review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text.replace('\n',''))
            else:
                review_dict['review'].append(review.find('div',class_='review_body').text.replace('\n',''))        

100%|████████████████████████████████████████████████████████████████████████████████| 278/278 [29:37<00:00,  6.39s/it]


In [64]:
reviews = pd.DataFrame(review_dict)
reviews['score'] = [int(x) for x in reviews['score']]
reviews['rating'] = ['Negative' if x < 5 else 'Positive' if x > 7
                     else 'Average' for x in reviews['score']]
reviews.to_csv('reviews.csv')